In [2]:
! pip install -U sentence-transformers


     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 757kB 8.8MB/s 
     |████████████████████████████████| 890kB 28.6MB/s 
     |████████████████████████████████| 3.0MB 37.4MB/s 
     |████████████████████████████████| 1.1MB 38.0MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.6.1-cp36-none-any.whl size=74031 sha256=e87a2e49f8e7e774b17b2ae247a7c8c908d5cddb9c448f08d60c6649b931a2aa
  Stored in directory: /root/.cache/pip/wheels/d7/fa/17/2b081a8cd8b0a86753fb0e9826b3cc19f0207062c0b2da7008
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=5b23293f7e69b737d9ee034b76ca6961f5d87b8f01083097bb9460791dd23614
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [3]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb 
import pickle

In [4]:
config = ppb.DistilBertConfig(max_position_embeddings=512)
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel(config), ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
# cd content/

/content


In [7]:
# cd drive/My\ Drive/raw_data/  

/content/drive/My Drive/raw_data


In [8]:
with open("Onehack.pickle", 'rb') as pickle_file:
  data = pickle.load(pickle_file)

In [12]:
# label=[]
tokens=[]
category=[]
batch_num=0
tokenout=[]

for batch in range(0,len(data)-100,100):  
  print(batch)
  batch_num+=1
  # label=[]
  tokens=[]
  # category=[]
  for topic in range(batch,batch+100):

    # label.append(slug[topic]['slug'])
    tokens.append(tokenizer.encode(data[topic][-1], max_length=512,pad_to_max_length=True,truncation=True,add_special_tokens=True))
    category.append(data[topic][1])
    if (topic == batch+99):

      tokenout.append(tokens)
      input_ids = torch.tensor(np.array(tokens))

      with torch.no_grad():
        last_hidden_states = model(input_ids)

      if (batch_num==1):
        features = last_hidden_states[0][:,0,:].numpy()
      
      else:
        features = np.concatenate((features,last_hidden_states[0][:,0,:].numpy()),axis=0)
      
      print("Done with Batch:",batch_num)
      
with open("Onehack_features.pickle", "wb") as output_file:
  pickle.dump(features, output_file)

#label = np.asarray(label).reshape(-1,1)
category = np.asarray(category).reshape(-1,1)
tokens = np.asarray(tokens)

pd.DataFrame(tokenout).to_csv("Onehack_tokens.csv")
#pd.DataFrame(label).to_csv("bnzlabels.csv")
pd.DataFrame(category).to_csv("Onehack_category.csv")
      

0
Done with Batch: 1
100
Done with Batch: 2
200
Done with Batch: 3
300
Done with Batch: 4
400
Done with Batch: 5
500
Done with Batch: 6
600
Done with Batch: 7
700
Done with Batch: 8
800
Done with Batch: 9
900
Done with Batch: 10
1000
Done with Batch: 11
1100
Done with Batch: 12
1200
Done with Batch: 13
1300
Done with Batch: 14
1400
Done with Batch: 15
1500
Done with Batch: 16
1600
Done with Batch: 17
1700
Done with Batch: 18
1800
Done with Batch: 19
1900
Done with Batch: 20
2000
Done with Batch: 21
2100
Done with Batch: 22
2200
Done with Batch: 23
2300
Done with Batch: 24
2400
Done with Batch: 25
2500
Done with Batch: 26
2600
Done with Batch: 27
2700
Done with Batch: 28
2800
Done with Batch: 29
2900
Done with Batch: 30
3000
Done with Batch: 31
3100
Done with Batch: 32
3200
Done with Batch: 33
3300
Done with Batch: 34
3400
Done with Batch: 35
3500
Done with Batch: 36
3600
Done with Batch: 37
3700
Done with Batch: 38
3800
Done with Batch: 39
3900
Done with Batch: 40
4000
Done with Batch:

In [13]:
from scipy.spatial.distance import cdist
Y = cdist(features, features, 'cosine')

In [14]:
# cd ..

/content/drive/My Drive


In [15]:
# cd raw_data/

/content/drive/My Drive/raw_data


In [16]:
with open("Onehack.pickle", 'rb') as pickle_file:
  text = pickle.load(pickle_file)

In [17]:
def recommend():
  val = int( input("Enter a number between 0 and 8400:"))
  print('')
  print("The article you've chosen:",data[val])

  similar_articles =  np.argsort(Y[val][:])[:11]
  print('')
  print("Few Articles you might like reading:")
  print('')
  for i in similar_articles[1:]:
    print(data[i])

In [18]:
recommend()

Enter a number between 0 and 800:1000

The article you've chosen: ['https://onehack.us/t/cmd-commands-massive-list-all-windows-users-hackers-should-know/89062', ' Tutorials &amp; Methods ', ['tutorial', 'hackers', 'cmd-commands', 'massive-list', 'windows-users'], 'Hello, OneHackers’Here I am introducing the most known and powerful CMD Command Prompts that every Noob or expert should know, to make your task easier that everyone does on a regular basis. Probably, a few of commands expertise already know, but let noobs take maximum advantage they can to become a pro, kindly proceed and start reading, Note: this list collected from specific resources, there is not accurate list order & thanks to all of them who written detailed description for specific commands.The Windows command prompt is a feature that’s been a core part of the Windows operating system for a long time. There are some CMD commands that are so useful and easy to use that even regular users see the Windows command prompt a

In [19]:
len(data)

8446